# **Building custom sources with SQL Databases**

We will be using dlt's `sql_database` source to build custom SQL sources

In [1]:
%%capture
!pip install pymysql duckdb dlt

`rest_api_source` -> Higher level, provides declarative way to configure sources

RestAPI Client -> Lower level, provides more granular control

## 1. Load Data From SQL DB

Fetch data from an open MySQL database and load it to DuckDB

```
# simply calling the sql_database() will load all tables from the DB
sql_database()

# use table_names arguement to narrow down tables
sql_database(table_names=["table1", "table2"])
```

In [2]:
import dlt
from dlt.sources.sql_database import sql_database

source = sql_database(
    "mysql+pymysql://rfamro@mysql-rfam-public.ebi.ac.uk:4497/Rfam",
    table_names=["family"]
)

pipeline = dlt.pipeline(
    pipeline_name="sql_database_example",
    destination="duckdb",
    dataset_name="sql_data",
    dev_mode=True,
)

load_info = pipeline.run(source)
print(load_info)

Pipeline sql_database_example load step completed in 2.68 seconds
1 load package(s) were loaded to destination duckdb and into dataset sql_data_20250422035424
The duckdb destination used duckdb:////content/sql_database_example.duckdb location to store data
Load package 1745337264.7823727 is LOADED and contains no failed jobs


### 1.1. Using the `query_adapter_callback` Argument

Callable to override the SELECT query used to fetch data from the table.

In [12]:
from sqlalchemy import text

# instead of doing a SELECT * FROM table, we are specifying here a filter condition
def query_adapter_callback(query, table, incremental=None, engine=None):
    return text(f"SELECT * FROM {table.fullname} WHERE rfam_id like '%bacteria%'")


# set the credendtials as ENV var to avoid declaring it again
import os
os.environ["SOURCES__SQL_DATABASE__CREDENTIALS"] = "mysql+pymysql://rfamro@mysql-rfam-public.ebi.ac.uk:4497/Rfam"


filtered_resource = sql_database(
    query_adapter_callback=query_adapter_callback,
    table_names=["family"]
)

info = pipeline.run(filtered_resource, table_name="bacterias")
print(info)

Pipeline sql_database_example load step completed in 0.20 seconds
1 load package(s) were loaded to destination duckdb and into dataset sql_data_20250422035424
The duckdb destination used duckdb:////content/sql_database_example.duckdb location to store data
Load package 1745338568.5666919 is LOADED and contains no failed jobs


In [16]:
print("Number of Rows: ", len(pipeline.dataset(dataset_type='default').bacterias.df()))

Number of Rows:  46


### 1.2. Using the `table_adapter_callback` Argument

It is used to modify the default schema of the table, adding columns or modifying the list of columns selected

In [17]:
# example 1 - adding a max_timestamp column to the tables for incremental loading

import sqlalchemy as sa
from sqlalchemy.sql import sqltypes

def add_max_timestamp(table):
  max_ts = sa.func.greatest(table.c.created, table.c.updated).label("max_timestamp")
  subq = sa.select(*table.c, max_ts).subquery()
  return subq


import dlt
from dlt.sources.sql_database import sql_table  # a dlt resource that loads a single table from the SQL database

table = sql_table(
    table="family",
    table_adapter_callback=add_max_timestamp,
    incremental=dlt.sources.incremental("max_timestamp")
)

info = pipeline.run(table, table_name = "family_with_max_ts")
print(info)

2025-04-22 16:17:16,180|[WARNING]|219|136695180943360|dlt|schema_types.py|sqla_col_to_column_schema:149|A column with name max_timestamp contains unknown data type NULL which cannot be mapped to `dlt` data type. When using sqlalchemy backend such data will be passed to the normalizer. In case of `pyarrow` and `pandas` backend, data types are detected from numpy ndarrays. In case of other backends, the behavior is backend-specific.
2025-04-22 16:17:16,243|[WARNING]|219|136695180943360|dlt|schema_types.py|sqla_col_to_column_schema:149|A column with name max_timestamp contains unknown data type NULL which cannot be mapped to `dlt` data type. When using sqlalchemy backend such data will be passed to the normalizer. In case of `pyarrow` and `pandas` backend, data types are detected from numpy ndarrays. In case of other backends, the behavior is backend-specific.


Pipeline sql_database_example load step completed in 0.17 seconds
1 load package(s) were loaded to destination duckdb and into dataset sql_data_20250422035424
The duckdb destination used duckdb:////content/sql_database_example.duckdb location to store data
Load package 1745338636.220226 is LOADED and contains no failed jobs


In [18]:
pipeline.dataset().family_with_max_ts.df().head()

,rfam_acc,rfam_id,auto_wiki,description,author,seed_source,gathering_cutoff,trusted_cutoff,noise_cutoff,comment,previous_id,cmbuild,cmcalibrate,cmsearch,num_seed,num_full,num_genome_seq,num_refseq,type,structure_source,number_of_species,number_3d_structures,num_pseudonokts,tax_seed,ecmli_lambda,ecmli_mu,ecmli_cal_db,ecmli_cal_hits,maxl,clen,match_pair_node,hmm_tau,hmm_lambda,created,updated,max_timestamp,_dlt_load_id,_dlt_id
0,RF00001,5S_rRNA,1302,5S ribosomal RNA,"Griffiths-Jones SR, Mifsud W, Gardner PP","Szymanski et al, 5S ribosomal database, PMID:1...",38.0,38.0,37.9,5S ribosomal RNA (5S rRNA) is a component of t...,None,cmbuild -F CM SEED,cmcalibrate --mpi CM,cmsearch --cpu 4 --verbose --nohmmonly -T 24.9...,712,594154,0,0,Gene; rRNA;,Published; PMID:11283358,12253,0,<NA>,,0.60889,-5.17319,1600000,225645,196,120,1,-3.7679,0.71820,2013-10-03 20:41:44+00:00,2024-09-09 21:15:12+00:00,2024-09-09 21:15:12+00:00,1745338539.3013937,Ideif+qwKqtoRA
1,RF00002,5_8S_rRNA,1303,5.8S ribosomal RNA,"Griffiths-Jones SR, Mifsud W","Wuyts et al, European LSU rRNA database, PMID:...",42.0,42.0,41.9,5.8S ribosomal RNA (5.8S rRNA) is a component ...,None,cmbuild -F CM SEED,cmcalibrate --mpi CM,cmsearch --cpu 4 --verbose --nohmmonly -T 19.6...,61,16501,0,0,Gene; rRNA;,Published; PMID:11125083,1591,0,<NA>,,0.66150,-9.18781,1600000,407997,265,154,1,-3.4977,0.71788,2013-10-03 20:47:00+00:00,2024-09-10 04:51:11+00:00,2024-09-10 04:51:11+00:00,1745338539.3013937,kedH1doK/7qTdg
2,RF00003,U1,1304,U1 spliceosomal RNA,"Griffiths-Jones SR, Mifsud W, Moxon SJ, Ontive...","Zwieb C, The uRNA database, PMID:9016512",66.0,66.0,65.9,U1 is a small nuclear RNA (snRNA) component of...,None,cmbuild -F CM SEED,cmcalibrate --mpi CM,cmsearch --cpu 4 --verbose --nohmmonly -T 25.0...,100,41297,0,0,Gene; snRNA; splicing;,Published; PMID:2405391,1539,0,<NA>,,0.66474,-8.65259,1600000,410919,262,166,1,-3.7311,0.71618,2013-10-03 20:57:11+00:00,2024-09-09 21:15:12+00:00,2024-09-09 21:15:12+00:00,1745338539.3013937,VuBdUZhAkL2b0A
3,RF00004,U2,1305,U2 spliceosomal RNA,"Griffiths-Jones SR, Mifsud W, Gardner PP","The uRNA database, PMID:9016512; Griffiths-Jon...",46.0,46.0,45.9,U2 is a small nuclear RNA (snRNA) component of...,None,cmbuild -F CM SEED,cmcalibrate --mpi CM,cmsearch --cpu 4 --verbose --nohmmonly -T 27.0...,208,71512,0,0,Gene; snRNA; splicing;,Published; PMID:2339054; Griffiths-Jones SR,2642,0,<NA>,,0.55201,-9.92571,1600000,404187,292,192,1,-3.4862,0.71304,2013-10-03 20:58:30+00:00,2024-09-10 04:51:11+00:00,2024-09-10 04:51:11+00:00,1745338539.3013937,7J0FJuR+F+zzIQ
4,RF00005,tRNA,1306,tRNA,"Eddy SR, Griffiths-Jones SR, Mifsud W",Eddy SR,29.0,29.0,28.9,Transfer RNA (tRNA) molecules are approximatel...,None,cmbuild -F CM SEED,cmcalibrate --mpi CM,cmsearch --cpu 4 --verbose --nohmmonly -T 22.0...,954,5335975,0,0,Gene; tRNA;,Published; PMID:8256282,14413,0,<NA>,,0.63376,-4.41829,1600000,281040,217,71,1,-2.6284,0.73562,2013-10-03 21:00:26+00:00,2024-09-10 04:51:11+00:00,2024-09-10 04:51:11+00:00,1745338539.3013937,w6vSExTztGeoCg


### 1.3. Using the `type_adapter_callback` Argument

When the default types don’t match what you want in the destination, you can remap them.


In [22]:
# check the existing schema

schema = pipeline.default_schema.to_dict()["tables"]["family"]["columns"]

for col in schema:
  print(schema[col]["name"], " : ", schema[col]["data_type"])

rfam_acc  :  text
rfam_id  :  text
auto_wiki  :  bigint
description  :  text
author  :  text
seed_source  :  text
gathering_cutoff  :  decimal
trusted_cutoff  :  decimal
noise_cutoff  :  decimal
comment  :  text
previous_id  :  text
cmbuild  :  text
cmcalibrate  :  text
cmsearch  :  text
num_seed  :  bigint
num_full  :  bigint
num_genome_seq  :  bigint
num_refseq  :  bigint
type  :  text
structure_source  :  text
number_of_species  :  bigint
number_3d_structures  :  bigint
num_pseudonokts  :  bigint
tax_seed  :  text
ecmli_lambda  :  decimal
ecmli_mu  :  decimal
ecmli_cal_db  :  bigint
ecmli_cal_hits  :  bigint
maxl  :  bigint
clen  :  bigint
match_pair_node  :  bigint
hmm_tau  :  decimal
hmm_lambda  :  decimal
created  :  timestamp
updated  :  timestamp
_dlt_load_id  :  text
_dlt_id  :  text


*Lets change columns from decimal to double, as it provides a greater numeric range which is better for scientific values*

In [23]:
import sqlalchemy as sa

def type_adapter_callback(sql_type):
  # if it is an instance of Numeric then return double type
  if isinstance(sql_type, sa.Numeric):
    return sa.Double
  return sql_type

new_source = sql_database(
    type_adapter_callback=type_adapter_callback,
    table_names=["family"]
)

info = pipeline.run(new_source, table_name="type_changed_family")
print(info)

Pipeline sql_database_example load step completed in 2.70 seconds
1 load package(s) were loaded to destination duckdb and into dataset sql_data_20250422035424
The duckdb destination used duckdb:////content/sql_database_example.duckdb location to store data
Load package 1745339645.0718708 is LOADED and contains no failed jobs


In [25]:
schema1 = pipeline.default_schema.to_dict()["tables"]["family"]["columns"]
schema2 = pipeline.default_schema.to_dict()["tables"]["type_changed_family"]["columns"]
column = "hmm_lambda"

print("For table 'family':", schema1[column]["name"], ":", schema1[column]["data_type"])
print("For table 'type_changed_family':", schema2[column]["name"], ":", schema2[column]["data_type"])

For table 'family': hmm_lambda : decimal
For table 'type_changed_family': hmm_lambda : double


In [26]:
decimal_columns = []
for col in schema1:
  if schema1[col]["data_type"] == "decimal":
    decimal_columns.append(schema1[col]["name"])

for col in schema2:
  if schema2[col]["name"] in decimal_columns:
    print(schema2[col]["name"])

gathering_cutoff
trusted_cutoff
noise_cutoff
ecmli_lambda
ecmli_mu
hmm_tau
hmm_lambda


### 1.4. Incremental Loading with `sql_database()` Source

dlt stores the pipeline state as a Python dictionary;

you can store values in it and, on the next pipeline run, request them back

In [27]:
import json
# sql_database_example was the name of one of the dlt pipelines created above

with open("/var/dlt/pipelines/sql_database_example/state.json", "r") as f:
  data = json.load(f)

data["sources"]["sql_database"]["resources"]["family"]["incremental"].keys()

dict_keys(['max_timestamp'])

In [28]:
# lets create a new key for incremental loading

from dlt.sources.sql_database import sql_database
import pendulum

source = sql_database().with_resources("family")
source.family.apply_hints(
    incremental=dlt.sources.incremental("updated", initial_value=pendulum.datetime(2024, 1, 1))
)

info = pipeline.run(source)
print(info)

2025-04-22 16:47:45,021|[WARNING]|219|136695180943360|dlt|__init__.py|_check_duplicate_cursor_threshold:591|Large number of records (832) sharing the same value of cursor field 'updated'. This can happen if the cursor field has a low resolution (e.g., only stores dates without times), causing many records to share the same cursor value. Consider using a cursor column with higher resolution to reduce the deduplication state size.


Pipeline sql_database_example load step completed in 2.28 seconds
1 load package(s) were loaded to destination duckdb and into dataset sql_data_20250422035424
The duckdb destination used duckdb:////content/sql_database_example.duckdb location to store data
Load package 1745340461.3747618 is LOADED and contains no failed jobs


In [29]:
import json
with open("/var/dlt/pipelines/sql_database_example/state.json", "r") as f:
  data = json.load(f)

data["sources"]["sql_database"]["resources"]["family"]["incremental"].keys()

dict_keys(['max_timestamp', 'updated'])

*We can see that two keys have been added now*